### 数据的加载

In [1]:
import sys
import pickle
sys.path.append("C:/Users/Administrator/ud120-projects-master/tools/")
sys.path.append("C:/Users/Administrator/ud120-projects-master/choose_your_own")
sys.path.append("C:/Users/Administrator/ud120-projects-master/final_project/")

import os
os.chdir('C:/Users/Administrator/ud120-projects-master/final_project')

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

### 数据预览

In [3]:
import pandas as pd
df = pd.DataFrame(data_dict)
df = df.T   #数据框转置
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,fraction_from_poi,fraction_to_poi,from_messages,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
ALLEN PHILLIP K,4175000,2869717,-3081055,NaN,phillip.allen@enron.com,1729541,13868,0.0161957,0.0296128,2195,...,304805,152,False,126027,-126027,201955,1407,2902,4484442,1729541
BADUM JAMES P,NaN,178980,NaN,NaN,NaN,257817,3486,0,0,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,182466,257817
BANNANTINE JAMES M,NaN,NaN,-5104,NaN,james.bannantine@enron.com,4046157,56301,0.0689046,0,29,...,NaN,864523,False,1757552,-560222,477,465,566,916197,5243487
BAXTER JOHN C,1200000,1295738,-1386055,NaN,NaN,6680544,11200,0,0,NaN,...,1586055,2660303,False,3942714,NaN,267102,NaN,NaN,5634343,10623258
BAY FRANKLIN R,400000,260455,-201641,NaN,frank.bay@enron.com,NaN,129142,0,0,NaN,...,NaN,69,False,145796,-82782,239671,NaN,NaN,827696,63014


### 数据探索

In [4]:
print 'Number of persons :{0}'.format(len(df))
print 'Number of features : {0}'.format(len(data_dict.values()[0]))
print 'Number of data points:', len(df.index) * len(df.columns)
print 'Number of POI:', len(df['poi'][df['poi'] == True])
print 'Number of not POI:', len(df['poi'][df['poi'] == False])

Number of persons :145
Number of features : 23
Number of data points: 3335
Number of POI: 18
Number of not POI: 127


### 数据清洗

In [5]:
print(df.describe())

       bonus deferral_payments deferred_income director_fees email_address  \
count    145               145             145           145           145   
unique    41                39              44            17           112   
top      NaN               NaN             NaN           NaN           NaN   
freq      64               107              97           129            34   

       exercised_stock_options expenses  fraction_from_poi  fraction_to_poi  \
count                      145      145              145.0            145.0   
unique                     101       94               75.0             62.0   
top                        NaN      NaN                0.0              0.0   
freq                        44       51               71.0             79.0   

       from_messages        ...        long_term_incentive other    poi  \
count            145        ...                        145   145    145   
unique            65        ...                         52    9

可以看出除了poi特征的top是False，其他特征的top都是NaN,特征的缺失值比较多。  
在Python中用NaN来标记缺失值，值为NaN的数据均不参与如求和、计数类的运算，后面只是对NaN进行描述性统计，对部分特征的NaN进行处理。

In [6]:
# 统计数据集中 NaN 的情况
import numpy as np
df.replace('NaN', np.nan, inplace = True)
features_with_nan = df.isnull().sum().sort_values(ascending=False)
person_with_nan = df.isnull().sum(axis = 1).sort_values(ascending=False)  #按clumns lables 排序

In [7]:
# 打印出缺失值最多的前 10位
print '\nTop 10 features_with_nan\n', features_with_nan[:10]
print '\nTop 10 person_with_nan\n', person_with_nan[:10]


Top 10 features_with_nan
loan_advances                142
director_fees                129
restricted_stock_deferred    128
deferral_payments            107
deferred_income               97
long_term_incentive           80
bonus                         64
from_messages                 59
from_poi_to_this_person       59
from_this_person_to_poi       59
dtype: int64

Top 10 person_with_nan
LOCKHART EUGENE E                20
WROBEL BRUCE                     18
WHALEY DAVID A                   18
GRAMM WENDY L                    18
THE TRAVEL AGENCY IN THE PARK    18
GILLIS JOHN                      17
WODRASKA JOHN                    17
SAVAGE FRANK                     17
SCRIMSHAW MATTHEW                17
CLINE KENNETH W                  17
dtype: int64


**缺失值** ： 
- 可以看出loan_advances 缺失值有142，只有4条记录。该特征不具有代表性，将该特征删除。  
- person里面的缺失值较多，其中LOCKHART EUGENE E，只有一条数据即poi，其他person中缺失值也较多。

**异常值**：

- 把person中THE TRAVEL AGENCY IN THE PARK 删除，明显不是人名
- 删除TOTAL,为财务数据的总和


 


In [8]:
# 特征的处理
df = df.drop(['TOTAL', 'LOCKHART EUGENE E', 'THE TRAVEL AGENCY IN THE PARK'], axis = 0)
df = df.drop(['email_address'], axis = 1)     #email_address 不具有唯一性

ValueError: labels ['TOTAL'] not contained in axis

### 特征工程  
 **[label]**： 
- 1. poi 是否是嫌疑人 (True=POI, Flase=Non-POI)  
**[邮件]**：
- 2. email_address          : 电子邮件地址
- 3. to_messages            : 发送的电子邮件数量
- 4. from_messages          : 收到的电子邮件数量
- 5. from_poi_to_this_person: 收到的来自 POI 的邮件数量
- 6. from_this_person_to_poi: 发送给 POI 的邮件数量
- 7. shared_receipt_with_poi: 收到的来自 POI 的抄送邮件  
**[财务]**：
- 8. salary                    : 工资
- 9. bonus                     : 奖金
- 10. long_term_incentive      : 长期激励薪酬  
- 11. deferred_income          : 延递收入  
- 12. deferral_payments        : 延期薪酬  
- 13. loan_advances            : 预支贷款  
- 14. other  
- 15. expenses                 : 开支  
- 16. director_fees            : 董事费  
- 17. total_payments           : 总薪酬和奖金  
- 18. exercised_stock_options  : 已行使过的股票期权
- 19. restricted_stock         : 受限股票
- 20. restricted_stock_deferred: 延期的受限股票
- 21. total_stock_value        : 总股票价值

### 建立新特征

In [ ]:
df['fraction_from_poi'] = df['from_poi_to_this_person'] / df['to_messages']
df['fraction_to_poi'] = df['from_this_person_to_poi'] / df['from_messages']

In [ ]:
df.head()

显示新增了2个字段，共22个字段

In [10]:
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
from tester import test_classifier

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list = ['poi','salary','bonus','long_term_incentive','deferred_income',
                 'deferral_payments','total_payments','exercised_stock_options',
                 'restricted_stock','restricted_stock_deferred','total_stock_value'] # You will need to use more features

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

### Task 2: Remove outliers
### Task 3: Create new feature(s)

### Store to my_dataset for easy export below.
data_dict.pop("TOTAL",0)

for i in data_dict:
    person = data_dict[i]
    if (all([person['from_poi_to_this_person'] != 'NaN',
             person['from_this_person_to_poi'] != 'NaN',
             person['to_messages'] != 'NaN',
             person['from_messages'] != 'NaN'])):
        fraction_from_poi = float(person["from_poi_to_this_person"]) / float(person["to_messages"])
        person["fraction_from_poi"] = fraction_from_poi
        fraction_to_poi = float(person["from_this_person_to_poi"]) / float(person["from_messages"])
        person["fraction_to_poi"] = fraction_to_poi
    else:
        person["fraction_from_poi"] = person["fraction_to_poi"] = 0

my_features_list = ['poi','salary','bonus','long_term_incentive','deferred_income',
                    'deferral_payments','total_payments','exercised_stock_options',
                    'restricted_stock','restricted_stock_deferred','total_stock_value',
                    'fraction_from_poi','fraction_to_poi']

my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, my_features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)


### 特征选择

In [11]:
from sklearn.feature_selection import SelectKBest
selector = SelectKBest(k='all').fit(features,labels)
selector.transform(features) 
selector.scores_

array([ 18.57570327,  21.06000171,  10.07245453,  11.59554766,
         0.21705893,   8.86672154,  25.09754153,   9.34670079,
         0.06498431,  24.46765405,   3.21076192,  16.64170707])

选择得分10分以上的，选择k=7

In [12]:
from sklearn.decomposition import PCA
pca=PCA(n_components=None)
pca.fit_transform(features)
pca.explained_variance_ratio_

array([  7.90181112e-01,   1.64856450e-01,   1.99246216e-02,
         1.01101602e-02,   7.32937146e-03,   4.28014808e-03,
         2.07746544e-03,   1.06996431e-03,   1.12390967e-04,
         5.83165845e-05,   2.06153327e-16,   6.38784948e-18])

创建了2个新的特征，fraction_from_poi(from_poi_to_this_person/to_messages)和fraction_to_poi(from_this_person_to_poi/from_messages),  
这两个新的特征反映的是与POI邮件来往的频率。加进my_features_list总共有13个特征，用PCA进行降维，PCA(n_components=3)时，pca.explained_variance_ratio_为[ 0.79018111  0.16485645  0.01992462]，最后一个能够对该数据集解释的概率为0.0199，  
故考虑PCA(n_components=2)，此时pca.explained_variance_ratio为[ 0.79018111  0.16485645]，此时特征为2维特征。

### 算法

In [13]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.

from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA

# Provided to give you a starting point. Try a variety of classifiers.


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

#LogisticRegression
pipe_lr = Pipeline([('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', LogisticRegression())])   #random_state=1
pipe_lr.fit(features_train, labels_train)
print('Test accuracy_lr: %.3f' % pipe_lr.score(features_test, labels_test))
test_classifier(pipe_lr, my_dataset, my_features_list)

#GaussianNB
from sklearn.naive_bayes import GaussianNB
pipe_nb = Pipeline([('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', GaussianNB()) ])
pipe_nb.fit(features_train, labels_train)
print('Test accuracy_nb: %.3f' % pipe_nb.score(features_test, labels_test))
test_classifier(pipe_nb, my_dataset, my_features_list)

#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
pipe_tr = Pipeline([ ('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', tree)])
pipe_tr.fit(features_train, labels_train)
print('Test accuracy_tr: %.3f' % pipe_tr.score(features_test, labels_test))
test_classifier(pipe_tr, my_dataset, my_features_list)

#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()   #max_depth=2, random_state=0
pipe_rf = Pipeline([('pca', PCA(n_components=2)),
                    ('clf', clf)])
pipe_rf.fit(features_train, labels_train)
print('Test accuracy_rf: %.3f' % pipe_rf.score(features_test, labels_test))
test_classifier(pipe_rf, my_dataset, my_features_list)

Test accuracy_lr: 0.909
Pipeline(memory=None,
     steps=[('sc', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])
	Accuracy: 0.87327	Precision: 0.58347	Recall: 0.17300	F1: 0.26687	F2: 0.20133
	Total predictions: 15000	True positives:  346	False positives:  247	False negatives: 1654	True negatives: 12753

Test accuracy_nb: 0.864
Pipeline(memory=None,
     steps=[('sc', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', GaussianNB(priors=None

#### 没有加新的特征的得分

In [14]:
#LogisticRegression
pipe_lr = Pipeline([('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', LogisticRegression())])   #random_state=1
pipe_lr.fit(features_train, labels_train)
test_classifier(pipe_lr, my_dataset, features_list)

#GaussianNB
from sklearn.naive_bayes import GaussianNB
pipe_nb = Pipeline([('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', GaussianNB()) ])
pipe_nb.fit(features_train, labels_train)
test_classifier(pipe_nb, my_dataset, features_list)

#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
pipe_tr = Pipeline([ ('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', tree)])
pipe_tr.fit(features_train, labels_train)
test_classifier(pipe_tr, my_dataset, features_list)

#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()   #max_depth=2, random_state=0
pipe_rf = Pipeline([('pca', PCA(n_components=2)),
                    ('clf', clf)])
pipe_rf.fit(features_train, labels_train)
test_classifier(pipe_rf, my_dataset, features_list)

Pipeline(memory=None,
     steps=[('sc', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])
	Accuracy: 0.86187	Precision: 0.45300	Recall: 0.17350	F1: 0.25090	F2: 0.19792
	Total predictions: 15000	True positives:  347	False positives:  419	False negatives: 1653	True negatives: 12581

Pipeline(memory=None,
     steps=[('sc', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', GaussianNB(priors=None))])
	Accuracy: 0.85927	Precision: 0.46094	Recal

||precision|recall|
|---|---|---|
|LogisticRegression|0.58347|0.17300|
|没加新特征|0.45300|0.17350|
|GaussianNB|0.44684|0.33200|
|没加新特征|0.46094|0.18400|
|DecisionTreeClassifier|0.26048|0.22050|
|没加新特征|0.19207|0.18400|
|RandomForestClassifier|0.25532|0.10800|
|没加新特征|0.27126|0.11800|

把seleckbest加入算法中

In [15]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.

from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA

# Provided to give you a starting point. Try a variety of classifiers.


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

#LogisticRegression
pipe_lr = Pipeline([('univ_select',SelectKBest(k=7)),
                    ('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', LogisticRegression())])   #random_state=1
pipe_lr.fit(features_train, labels_train)
test_classifier(pipe_lr, my_dataset, my_features_list)

#GaussianNB
from sklearn.naive_bayes import GaussianNB
pipe_nb = Pipeline([('univ_select',SelectKBest(k=7)),
                    ('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', GaussianNB())])
pipe_nb.fit(features_train, labels_train)
test_classifier(pipe_nb, my_dataset, my_features_list)

#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
pipe_tr = Pipeline([('univ_select',SelectKBest(k=7)),
                    ('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', tree)])
pipe_tr.fit(features_train, labels_train)
test_classifier(pipe_tr, my_dataset, my_features_list)

#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()   #max_depth=2, random_state=0
pipe_rf = Pipeline([('univ_select',SelectKBest(k=7)),
                    ('pca', PCA(n_components=2)),
                    ('clf', clf)])
pipe_rf.fit(features_train, labels_train)
test_classifier(pipe_rf, my_dataset, my_features_list)

Pipeline(memory=None,
     steps=[('univ_select', SelectKBest(k=7, score_func=<function f_classif at 0x00000000080D5198>)), ('sc', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', Lo...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])
	Accuracy: 0.86320	Precision: 0.46049	Recall: 0.15150	F1: 0.22799	F2: 0.17498
	Total predictions: 15000	True positives:  303	False positives:  355	False negatives: 1697	True negatives: 12645

Pipeline(memory=None,
     steps=[('univ_select', SelectKBest(k=7, score_func=<function f_classif at 0x00000000080D5198>)), ('sc', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', GaussianNB(priors=None))])
	Accuracy: 0.85160	Preci

|Algorithm  | Accuracy |Precision |Recall|F1 |  F2|
|-----         |------|-----|----|-----|-----|
|LogisticRegression    |0.87327   |0.58347   | 0.17300|0.26687  |0.20133|
|增加selectkbest|0.86320|0.46049|0.15150|0.22799|0.17498|
|GaussianNB      |0.85613    | 0.44684  |0.33200 | 0.38095|0.34999|
|增加selectkbest|0.85160| 0.41281|0.26750|0.32464|0.28776|
|DecisionTreeClassifier|0.81153  |0.25691  |0.21850|0.23615  |0.22523|
|增加selectkbest|0.82140|0.32028|0.30250|0.31113|0.30590|
|RandomForestClassifier|0.83827|0.25175|0.10800| 0.15115|0.12192|
|增加selectkbest|0.84873|0.34871|0.15500|0.21461|0.17437|

4种算法中加入selectkbest后，其中LogisticRegression和GaussianNB效果没有不加好，  
而DecisionTreeClassifier和RandomForestClassifier增加之后，各部分得分都明显提高。

缩放特征，选择SVC算法

In [18]:
 from sklearn import svm
clf = svm.SVC()   
clf.fit(features_train, labels_train)
test_classifier(clf, my_dataset, my_features_list)

Got a divide by zero when trying out: SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Precision or recall may be undefined due to a lack of true positive predicitons.


SVC效果不是很好

In [17]:
# KNN算法
from sklearn.neighbors.nearest_centroid import NearestCentroid
clf = NearestCentroid()
clf.fit(features_train, labels_train)
test_classifier(clf, my_dataset, my_features_list)

NearestCentroid(metric='euclidean', shrink_threshold=None)
	Accuracy: 0.83680	Precision: 0.36035	Recall: 0.28900	F1: 0.32075	F2: 0.30092
	Total predictions: 15000	True positives:  578	False positives: 1026	False negatives: 1422	True negatives: 11974



In [19]:
clf = NearestCentroid()
pipe_knn = Pipeline([('univ_select',SelectKBest(k=7)),
                    ('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', NearestCentroid())])
pipe_knn.fit(features_train, labels_train)
test_classifier(pipe_knn, my_dataset, my_features_list)

Pipeline(memory=None,
     steps=[('univ_select', SelectKBest(k=7, score_func=<function f_classif at 0x00000000080D5198>)), ('sc', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', NearestCentroid(metric='euclidean', shrink_threshold=None))])
	Accuracy: 0.83247	Precision: 0.38544	Recall: 0.43150	F1: 0.40717	F2: 0.42143
	Total predictions: 15000	True positives:  863	False positives: 1376	False negatives: 1137	True negatives: 11624



调参之后KNN算法各项指标提高了很多

### 调参

In [84]:
# LogisticRegression
param_grid =[{'penalty': ('l1', 'l2'),
               'C': [0.1, 1.0, 10, 100]}]

lr = GridSearchCV( LogisticRegression(),
                  param_grid = param_grid, scoring = 'f1', cv = 10)
lr.fit(features_train, labels_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'penalty': ('l1', 'l2'), 'C': [0.1, 1.0, 10, 100]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [86]:
lr_best = lr.best_estimator_ #LogisticRegression
print lr_best

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


#### LogisticRegression 调参结果

In [87]:
test_classifier(lr_best, my_dataset, my_features_list)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
	Accuracy: 0.86220	Precision: 0.47079	Recall: 0.27000	F1: 0.34318	F2: 0.29518
	Total predictions: 15000	True positives:  540	False positives:  607	False negatives: 1460	True negatives: 12393



In [79]:
#RandomForestClassifier
param_grid =[{ 'n_estimators': [5, 10, 50],
               'criterion': ('gini', 'entropy'), 
               'min_samples_split': [2, 5, 10, 20, 40, 70]}]

rf = GridSearchCV( RandomForestClassifier(class_weight='balanced', random_state=42),
                  param_grid = param_grid, scoring = 'f1', cv = 10)
rf.fit(features_train, labels_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [5, 10, 50], 'min_samples_split': [2, 5, 10, 20, 40, 70], 'criterion': ('gini', 'entropy')}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [83]:
rf_best = rf.best_estimator_ # RandomForestClassifier
print rf_best

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)


#### RandomForestClassifier 调参结果

In [82]:
test_classifier(rf_best, my_dataset, my_features_list)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)
	Accuracy: 0.80680	Precision: 0.33209	Recall: 0.44400	F1: 0.37997	F2: 0.41596
	Total predictions: 15000	True positives:  888	False positives: 1786	False negatives: 1112	True negatives: 11214



In [77]:
#DecisionTreeClassifier
param_grid =[{ 'splitter': ('best', 'random'),
               'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
               'max_depth': [None, 10, 30, 50, 70, 100] }]

tr = GridSearchCV( DecisionTreeClassifier(),
                  param_grid = param_grid, scoring = 'f1', cv = 10)
tr.fit(features_train, labels_train)
test_classifier(tr, my_dataset, my_features_list)

GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'splitter': ('best', 'random'), 'max_depth': [None, 10, 30, 50, 70, 100]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)
	Accuracy: 0.82233	Precision: 0.31456	Recall: 0.28200	F1: 0.29739	F2: 0.28796
	Total predictions: 15000	True positives:  564	False positives: 1229	False negatives: 1436	True negatives: 11771



In [88]:
#DecisionTreeClassifier
param_grid =[{ 'splitter': ('best', 'random'),
               'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
               'max_depth': [None, 10, 30, 50, 70, 100] }]

tr = GridSearchCV( DecisionTreeClassifier(),
                  param_grid = param_grid, scoring = 'f1', cv = 10)
tr.fit(features_train, labels_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'splitter': ('best', 'random'), 'max_depth': [None, 10, 30, 50, 70, 100]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [90]:
tr_best = tr.best_estimator_ # DecisionTreeClassifier
print tr_best

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='random')


#### DecisionTreeClassifier调参结果

In [91]:
test_classifier(tr_best, my_dataset, my_features_list)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='random')
	Accuracy: 0.80600	Precision: 0.30200	Recall: 0.34700	F1: 0.32294	F2: 0.33696
	Total predictions: 15000	True positives:  694	False positives: 1604	False negatives: 1306	True negatives: 11396



In [ ]:
# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)